In [11]:

from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2')
# embeddings = embedding_model.encode(sentences)
input_query = "What is Metabolism of Proteins"
query_embedding = embedding_model.encode(input_query)
query_embedding_list = query_embedding.tolist()

print("query_embedding len", len(query_embedding_list))
print(query_embedding_list[:5])

query_embedding len 768
[0.05997343361377716, -0.013056949712336063, 0.023020777851343155, -0.02001880295574665, -0.007817837409675121]


In [13]:
print(type(query_embedding_list))
print(type(query_embedding))

<class 'list'>
<class 'numpy.ndarray'>


In [5]:
import psycopg2
import numpy as np
import os
from dotenv import load_dotenv

def connect_to_db():
    # Load environment variables from .env file
    load_dotenv()
    DB_NAME = os.getenv("DB_NAME")
    DB_USER = os.getenv("DB_USER")
    DB_PASS = os.getenv("DB_PASS")
    DB_HOST = os.getenv("DB_HOST")
    DB_PORT = os.getenv("DB_PORT")
    
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASS,
            host=DB_HOST,
            port=DB_PORT
        )
        print("Connected to PostgreSQL database successfully!")
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to PostgreSQL database: {e}")
        return None


In [ ]:

# CREATE OR REPLACE FUNCTION get_embedding(	
# 	rows_limit int,
#     embedding_input vector(768)
# )
# RETURNS TABLE (id bigint, chunk varchar(400), embedding vector(768))
# LANGUAGE plpgsql
# AS $$
# BEGIN    
# 	RETURN QUERY SELECT ni.id,ni.chunk, ni.embedding FROM nutritionitems ni
# 	ORDER BY ni.embedding <=> embedding_input LIMIT rows_limit;
# END;
# $$;


In [24]:
def fetch_data(conn):    
    try:
        with conn.cursor() as cur:
            select_sql = """SELECT get_embedding(%s,%s::vector);"""
            cur.execute(select_sql, (3, query_embedding_list))
            nutritionitems = cur.fetchall()            
            for user in nutritionitems:
                print(f"ID: {nutritionitems[0]}, Chunk: {nutritionitems[1]}, Embedding: {nutritionitems[2]}")
    except psycopg2.Error as e:
        print(f"Error fetching data: {e}")

conn = connect_to_db()
fetch_data(conn)
if conn:
    conn.close()
    print("Connection closed.")

Connected to PostgreSQL database successfully!
ID: ('(1250,"protein metabolism were proposed and  challenged, leading to the more or less contemporary  view which was established through the seminal work  of Rudolf Schoenheimer, conducted at Columbia","[0.052537404,0.005378502,0.021275485,-0.005041571,-0.002239069,0.00027202087,-0.048808005,0.012987208,-0.015270091,-0.069770165,0.0012154859,0.003150373,-0.026640443,-0.017754145,-0.00756407,0.034633297,-0.0045439145,-0.01080621,-0.041147348,0.035433408,-0.051737465,-0.016448043,0.030341962,-0.030173682,0.054849703,0.03906912,0.010409088,-0.034177117,-0.012022656,0.0017017266,-0.06477346,0.048235428,0.04838385,0.003910021,1.5243239e-06,-0.041973162,-0.033002235,0.02757624,-0.06790265,0.04787971,-0.020934356,0.007594279,-0.037245203,0.030626746,0.07125951,0.05422519,0.012824483,0.008682086,-0.07080558,-0.028458737,-0.01416838,-0.033069897,0.031268775,-0.01877296,0.03939771,0.017313503,0.015026457,0.06861323,-0.0010287714,0.00470658,0.0014